# Harmony PI 21.1 Sprint 1 Demo

In Sprint 1 of PI 21.1, Harmony added the following
1. allowing a request to specify collections by short name (HARMONY-229)
2. providing an indication of how many granules will be processed in the request response (HARMONY-306)
3. applying scaling and offsetting to the Zarr data returned (HARMONY-664)

This notebook provides a basic workflow to demonstrate this functrionality. For more a general introduction and tutorial, see [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb).  Useful helpers for making the calls found in this notebook can be found under the [docs/notebook-helpers](./notebook-helpers) folder.

## Prerequisites

1. Install Python 3. This notebook is tested to work in 3.8 but should work in most recent 3.x versions.
2. Install Jupyter: pip install jupyterlab
3. Setup your ~/.netrc for Earthdata Login as described in Harmony API Introduction

## Set Up AWS

Once you have Zarr links, you can access them with your AWS credentials to the Harmony account.  Obtain the credentials and make sure your default AWS account uses them.  One way to do this is to edit `~/.aws/credentials` to have the following section:
```
[default]
aws_access_key_id = YOUR_HARMONY_ACCESS_KEY_ID
aws_secret_access_key = YOUR_HARMONY_SECRET_ACCESS_KEY
```
Restart your Jupyter kernel after completing this step

## Setup notebook environment and imports

We need to set up general-purpose imports and authentication

In [ ]:
# %load_ext autoreload
# %autoreload
# %matplotlib inline

import sys
# Install dependencies into the Jupyter Kernel
!{sys.executable} -m pip install -q -r notebook_helpers/requirements.txt
!{sys.executable} -m pip install s3fs zarr

# Import libraries used throughout the notebook
import json
from pprint import pprint
from time import sleep
from notebook_helpers import get, post, show, get_data_urls, show_async, show_async_condensed, print_async_status, show_shape


## Request Subsetted Zarr Data Using ShortName

This request asks for variable subsetting of L2 data with output in the Zarr format.

In [ ]:
harmony_root = 'https://harmony.uat.earthdata.nasa.gov'
asyncConfig = {
    'short-name': 'harmony_example_l2',
    'ogc-api-coverages_version': '1.0.0',
    'variable': 'all',
    'format': 'application/x-zarr',
    'time': '("2020-01-10T00:00:00.000Z":"2020-01-13T00:00:00.000Z")',
    'max_results': '2'
}

async_url = harmony_root+'/{short-name}/ogc-api-coverages/{ogc-api-coverages_version}/collections/{variable}/coverage/rangeset?subset=time{time}&maxResults={max_results}&format={format}'.format(**asyncConfig)
async_response = get(async_url)

### Display numInputGranules field in response

In [ ]:
print(async_response.json()["numInputGranules"])

### Wait for results and get data links

In [ ]:
show_async_condensed(async_response, show_results=False)
job_results = get(async_response.url).json()
data_links = [link['href'] for link in job_results['links'] if link.get('rel') == 'data']


### Open a Zarr file

In [ ]:
import s3fs
import zarr

fs = s3fs.S3FileSystem(client_kwargs={'region_name':'us-west-2'})
store = fs.get_mapper(root=data_links[0], check=False)
zarr_file = zarr.open(store)
zarr_file['blue_var'][0][100]